# Hands-on example (Azure ML)

In [1]:
!python --version

Python 3.7.4


In [2]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

#Azure
import azureml.core
import azureml.dataprep
from azureml.core import Workspace, Dataset
from azure.storage.blob import BlobServiceClient
from azureml.core.authentication import ServicePrincipalAuthentication

#MLFlow
import mlflow
import mlflow.sklearn
from mlflow.entities import ViewType

#Temporarily filter all warning for demo
warnings.filterwarnings("ignore")

In [3]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.11.0


### Set Up Authentication

In [4]:
from settings import (AZURE_SUBSCRIPTION_ID,AZURE_RESOURCE_GROUP, 
                     AZURE_TENANT_ID)

In [5]:
subscription_id = AZURE_SUBSCRIPTION_ID
resource_group = AZURE_RESOURCE_GROUP
workspace_name = 'mlflow_tutorial'

## 0. The data

* The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
* P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
* Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

In [6]:
##Use this if inside AML Studio
# ws = Workspace.from_config()

ws = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(ws, name='wine-quality')
dataset.to_pandas_dataframe().head()

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


## 1. Tracking experiments

### Set Tracking URI to Azure ML Workspace

In [7]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [8]:
mlflow.tracking.get_tracking_uri()

'azureml://eastus.experiments.azureml.net/mlflow/v1.0/subscriptions/e1eb783d-78a5-42a4-bae3-bc0ddd433898/resourceGroups/robotdemo/providers/Microsoft.MachineLearningServices/workspaces/mlflow_tutorial?'

In [9]:
exp_name = "ElasticNet_wine_AML"
mlflow.set_experiment(exp_name)

### What do we track?

- **Code Version**: Git commit hash used for the run (if it was run from an MLflow Project)
- **Start & End Time**: Start and end time of the run
- **Source**: what code run?
- **Parameters**: Key-value input parameters.
- **Metrics**: Key-value metrics, where the value is numeric (can be updated over the run)
- **Artifacts**: Output files in any format.

In [10]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(dataset):
    data = dataset.to_pandas_dataframe()

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5,full_view=False):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(dataset)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        if full_view:
            print("Run IDs: \n{}".format(mlflow.search_runs(ViewType.ACTIVE_ONLY)))
        else:
            pass
        
        mlflow.sklearn.log_model(lr, "model")        

In [11]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8222428497595401
  MAE: 0.6278761410160693
  R2: 0.12678721972772666
Save to: azureml://experiments/ElasticNet_wine_AML/runs/381209a4-3b6e-4e32-a64d-4a152e4471a4/artifacts


2020/08/07 18:24:32 WARNING mlflow.models: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/ElasticNet_wine_AML/runs/381209a4-3b6e-4e32-a64d-4a152e4471a4/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


In [12]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.2022463182289208
Save to: azureml://experiments/ElasticNet_wine_AML/runs/0b4179d6-13c1-4a22-95e5-abbaf540be2f/artifacts


2020/08/07 18:25:18 WARNING mlflow.models: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/ElasticNet_wine_AML/runs/0b4179d6-13c1-4a22-95e5-abbaf540be2f/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


In [ ]:
train(0.1, 0.1,full_view=True)

### 1.1 Comparing runs
Connect to Azure Portal to view experiments and run metrics.

In [13]:
ws.experiments[exp_name]

Name,Workspace,Report Page,Docs Page
ElasticNet_wine_AML,mlflow_tutorial,Link to Azure Machine Learning studio,Link to Documentation


## 2. Tagging runs

In [21]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='', experiment_id='fe3ff3a4-0fa3-4075-bd4f-8352d3b501d9', lifecycle_stage='active', name='ElasticNet_wine_AML', tags={}>]


In [22]:
# get the run
_run = client.get_run(run_id="53254795-c368-4155-a04e-b570ca7d9e60")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6112547988118586,
 'r2': 0.2157063843066196,
 'rmse': 0.7792546522251949}, params={'alpha': '0.1', 'l1_ratio': '0.1'}, tags={'mlflow.source.git.commit': '5a96d9f2cd8b068a0be1baca9c7c77386f03d047',
 'mlflow.source.name': '/Users/antfra/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'antfra'}>, info=<RunInfo: artifact_uri='azureml://experiments/ElasticNet_wine_AML/runs/53254795-c368-4155-a04e-b570ca7d9e60/artifacts', end_time=1596808619296, experiment_id='fe3ff3a4-0fa3-4075-bd4f-8352d3b501d9', lifecycle_stage='active', run_id='53254795-c368-4155-a04e-b570ca7d9e60', run_uuid='53254795-c368-4155-a04e-b570ca7d9e60', start_time=1596808612524, status='FINISHED', user_id='c8427b67-4103-4ced-b0c1-2200b7c3dcfa'>>


In [14]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)